In [3]:
import numpy as np
import pandas as pd
import os
import pywt
from scipy.stats import entropy

In [4]:
# 🔹 File Paths
eeg_data_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/chbmit_preprocessed_data.csv"
seizure_info_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/sizuretimes.xlsx"

# 🔹 Output Folder (Create if not exists)
output_folder = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features"
os.makedirs(output_folder, exist_ok=True)

# 🔹 Load Patient List from Seizure Info Excel (Extract Sheet Names)
xls = pd.ExcelFile(seizure_info_path)
patient_ids = xls.sheet_names  # Extract patient IDs dynamically (e.g., "chb24", "chb23", ...)

In [5]:
# 🔹 Load Preprocessed EEG Data
def load_eeg_data():
    if not os.path.exists(eeg_data_path):
        print(f"❌ File not found: {eeg_data_path}")
        return None
    
    df = pd.read_csv(eeg_data_path)
    print("📂 EEG Data Loaded. Shape:", df.shape)

    return df

In [6]:
# 🔹 Function to Compute Wavelet Features
def compute_wavelet_features(eeg_signals, wavelet='db4', level=3):
    wavelet_features = {}

    for channel in eeg_signals.columns:
        signal = eeg_signals[channel].values

        # ✅ Apply Discrete Wavelet Transform (DWT)
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        
        # ✅ Extract Detail and Approximate Coefficients
        approx_coeffs = coeffs[0]  # Approximate (Low-frequency)
        detail_coeffs = np.concatenate(coeffs[1:])  # All Detail (High-frequency)

        # ✅ Compute Features
        wavelet_features[channel] = {
            "Approx Mean": np.mean(approx_coeffs),
            "Approx Std": np.std(approx_coeffs),
            "Approx Energy": np.sum(approx_coeffs ** 2),
            "Approx Entropy": entropy(np.abs(approx_coeffs) + 1e-10),  # Avoid log(0)
            "Detail Mean": np.mean(detail_coeffs),
            "Detail Std": np.std(detail_coeffs),
            "Detail Energy": np.sum(detail_coeffs ** 2),
            "Detail Entropy": entropy(np.abs(detail_coeffs) + 1e-10),
        }

    # Convert to DataFrame
    wavelet_df = pd.DataFrame.from_dict(wavelet_features, orient='index')
    return wavelet_df


In [7]:
# 🔹 Process Each Patient
df = load_eeg_data()
if df is not None:
    for patient in patient_ids:
        print(f"\n📂 Processing {patient}...")

        # ✅ Load seizure data for this patient
        seizure_df = pd.read_excel(seizure_info_path, sheet_name=patient, engine="openpyxl")

        # ✅ Check if `start` and `stop` columns exist
        if "start" not in seizure_df.columns or "stop " not in seizure_df.columns:
            print(f"⚠️ Skipping {patient}: 'start' or 'stop ' columns not found.")
            continue  # Skip if no valid columns

        # ✅ Extract seizure time rows
        patient_rows = seizure_df[['start', 'stop ']].dropna()
        patient_eeg_data = df.iloc[patient_rows.index, :-1]  # Select only relevant EEG signals

        # ✅ Compute Wavelet Features for this patient
        wavelet_features_df = compute_wavelet_features(patient_eeg_data)

        # 🔹 Save each patient's wavelet features as a separate CSV file
        output_file = os.path.join(output_folder, f"{patient}_wavelet_features.csv")
        wavelet_features_df.to_csv(output_file)

        print(f"✅ Saved: {output_file}")

print("\n✅ Wavelet Transform Feature Extraction Completed for All Patients!")


📂 EEG Data Loaded. Shape: (2097150, 24)

📂 Processing chb24...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb24_wavelet_features.csv

📂 Processing chb23...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb23_wavelet_features.csv

📂 Processing chb22...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb22_wavelet_features.csv

📂 Processing chb21...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb21_wavelet_features.csv

📂 Processing chb20...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb20_wavelet_features.csv

📂 Processing chb19...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb19_wavelet_features.csv

📂 Processing chb18...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb18_wavelet_features.csv

📂 Processing chb17...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb17_wavelet_features.csv

📂 Processing chb16...
✅ Saved: 

/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will e

✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb15_wavelet_features.csv

📂 Processing chb14...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb14_wavelet_features.csv

📂 Processing chb13...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb13_wavelet_features.csv

📂 Processing chb12...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb12_wavelet_features.csv

📂 Processing chb11...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb11_wavelet_features.csv

📂 Processing chb10...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb10_wavelet_features.csv

📂 Processing chb09...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb09_wavelet_features.csv

📂 Processing chb08...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb08_wavelet_features.csv

📂 Processing chb07...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Wavelet_Features/chb07_

/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(
